In [1]:
%matplotlib inline

import numpy as np
import pandas as pd
import torch
import matplotlib.pyplot as plt
plt.rcParams['figure.figsize'] = (16.0, 8.0)

from gluonts.nursery.ts_embeddings.run import MyDataModule

In [2]:
encoder_model_path = './encoder.pt'
print('loading encoder')
loaded_encoder = torch.jit.load(encoder_model_path)

loading encoder


In [3]:
multivar_dim = 1
dm = MyDataModule(ts_len=1000, dataset_name='m5', num_workers=4,
                 batch_size=128, multivar_dim=multivar_dim, shuffle=True)
dm.setup()

/home/kashif/.env/pytorch/lib/python3.8/site-packages/gluonts/dataset/common.py:323: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  timestamp = pd.Timestamp(string, freq=freq)
/home/kashif/.env/pytorch/lib/python3.8/site-packages/gluonts/dataset/common.py:326: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  if isinstance(timestamp.freq, Tick):
/home/kashif/.env/pytorch/lib/python3.8/site-packages/gluonts/dataset/common.py:328: FutureWarning: Timestamp.freq is deprecated and will be removed in a future version
  timestamp.floor(timestamp.freq), timestamp.freq
/home/kashif/.env/pytorch/lib/python3.8/site-packages/gluonts/dataset/common.py:327: FutureWarning: The 'freq' argument in Timestamp is deprecated and will be removed in a future version.
  return pd.Timestamp(


skipped 5702 series, because the target was smaller than ts_len=1000
num series 24788
skipped 5702 series, because the target was smaller than ts_len=1000
num series 24788
Shape of the tensor: torch.Size([24788, 1, 1000]) and we have 24788 time series of length 1000.


In [4]:
train_embeddings = []
train_label = []
for i, [batch,label] in enumerate(dm.train_dataloader()):
    with torch.no_grad():
        embed = loaded_encoder(batch)
    train_embeddings.append(embed)
    train_label.append(label[:, -1])
    if len(train_embeddings) > 500:
        break
        
train_embeddings = torch.cat(train_embeddings, dim=0)
train_label = torch.cat(train_label, dim=0)

In [5]:
test_embeddings = []
test_label = []
for i, [batch,label] in enumerate(dm.test_dataloader()):
    with torch.no_grad():
        embed = loaded_encoder(batch)
    test_embeddings.append(embed)
    test_label.append(label[:, -1])
    if len(test_embeddings) > 500:
        break
        
test_embeddings = torch.cat(test_embeddings, dim=0)
test_label = torch.cat(test_label, dim=0)

In [6]:
dist = torch.mm(test_embeddings, train_embeddings.T)

In [21]:
yd, yi = dist.topk(1, axis=1, largest=True, sorted=True)

In [22]:
retrieval = torch.gather(train_label, 0, yi.squeeze())

In [23]:
correct = retrieval.eq(test_label).sum().item()

In [24]:
total = test_label.size(0)

In [25]:
correct*100./total

9.885038860103627

In [32]:
yd, yi = dist.topk(200, dim=1, largest=True, sorted=True)

In [46]:
yi.shape

torch.Size([24704, 200])

In [49]:
train_label.unsqueeze(0).shape

torch.Size([1, 24704])

In [52]:
retrieval = torch.gather(train_label.unsqueeze(0), -1, yi)

RuntimeError: Size does not match at dimension 0 expected index [24704, 200] to be smaller than src [1, 24704] apart from dimension 1